# Seminar 10 - RecSys

## Imports

In [ ]:
import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

## The Movies Dataset
[Качать отсюдава](https://www.kaggle.com/rounakbanik/the-movies-dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/ML/семестр1/Семинар\ 10\ -\ RecSys/data

archive.zip  lastfm-dataset-1K	lastfm-dataset-1K.tar.gz  movies


In [ ]:
!unzip -q /content/gdrive/My\ Drive/Colab\ Notebooks/ML/семестр1/Семинар\ 10\ -\ RecSys/data/archive.zip \
              -d ./gdrive/My\ Drive/Colab\ Notebooks/ML/семестр1/Семинар\ 10\ -\ RecSys/data/movies

Content:
- movies_metadata.csv: This file contains information on ~45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, genre, revenue, release dates, languages, production countries, and companies.
- keywords.csv: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.
- credits.csv: Consists of Cast and Crew Information for all the movies. Available in the form of a stringified JSON Object.
- links.csv: This file contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.
- links_small.csv: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.
- ratings_small.csv: The subset of 100,000 ratings from 700 users on 9,000 movies.


In [ ]:
metadata = pd.read_csv('./gdrive/My Drive/Colab Notebooks/ML/семестр1/Семинар 10 - RecSys/data/movies/movies_metadata.csv', low_memory=False)

metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
metadata.shape

(45466, 24)

In [ ]:
rating = pd.read_csv('./gdrive/My Drive/Colab Notebooks/ML/семестр1/Семинар 10 - RecSys/data/movies/ratings_small.csv', low_memory=False)

rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## Неперсонализированная рекомендация

Сделаем свой аналог [IMDb rating](https://www.imdb.com/chart/top?ref_=nb_mv_3_chttp)

[Про взвешенный рейтинг](https://www.quora.com/How-does-IMDbs-rating-system-work)

$$WeightedRating=(\frac{v}{v+m}⋅R)+(\frac{m}{v+m}⋅C)$$

где:
- v (votes) число оценок фильма;

- m (minimum) минимальное число оценок для попадания в топ;

- R (rating) средний рейтинг фильма;

- C (across) средний рейтинг по всем фильмам.

In [ ]:
# Calculate mean of vote average column
C = 
print(C)

5.618207215133889


In [ ]:
# Calculate the minimum number of votes required to be in the chart, m
m = 
print(m)

160.0


In [ ]:
# Filter out all qualified movies into a new DataFrame
q_movies = 
q_movies.shape

(4555, 24)

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return 

In [ ]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(20)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


Получилось достаточно близко к оригинальному топу.


О чем это говорит?

## Content-based рекомендации

Сделаем рекомендательную систему на основе описания фильмов

In [ ]:
# Print plot overviews of the first 5 movies.
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [ ]:
firast_n =  metadata.copy()[:30000]

Что делать если мы не разбираемся в NLP?

### Найдем векторное представление данных

Просто посмотрим на вхождение слов в описание каждого из фильмов
Оценивать сходство фильмов будем с помощью [TF-IDF (Term Frequency-Inverse Document Frequency)](https://ru.wikipedia.org/wiki/TF-IDF) векторов

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

In [ ]:
#Replace NaN with an empty string
firast_n['overview'] = firast_n['overview'].fillna('')

In [ ]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(firast_n['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(30000, 5000)

In [ ]:
#Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[500:510]

### Оценим схожесть полученных векторов

Схожесть будем измерять по косинусной метрике

![](https://drive.google.com/uc?export=view&id=1jePvlnj5s0S7LTqWn6_p-eU5XUCfKgTq)

In [ ]:
# Compute the cosine similarity matrix
cosine_sim = 


In [ ]:
cosine_sim.shape

(30000, 30000)

In [ ]:
print(cosine_sim[0])

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(firast_n.index, index=firast_n['title']).drop_duplicates()

In [ ]:
indices

In [ ]:
# Function that takes in movie title as input and outputs most similar movies

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = 

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = 

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')

 Как учесть все предпочтения?

### Сделеаем рекомендацию для пользователя

In [ ]:
rating = rating.drop(['timestamp'], axis=1)
rating

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [ ]:
user_rating = 
user_rating

In [ ]:
user_rating["movieId"].values[0]

In [ ]:
firast_n["title"].values[1172]

In [ ]:
indices[]

In [ ]:
get_recommendations()

## LASTFM Dataset

А теперь будем рекомендовать музыку

[Качать отсюдава](http://ocelma.net/MusicRecommendationDataset/lastfm-1K.html)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import tarfile
lastfm = tarfile.open('./gdrive/My Drive/Colab Notebooks/ML/семестр1/Семинар 10 - RecSys/data/lastfm-dataset-1K.tar.gz', 'r')
lastfm.extractall('./gdrive/My Drive/Colab Notebooks/ML/семестр1/Семинар 10 - RecSys/data/')

In [ ]:
all_data = pd.read_csv('./gdrive/My Drive/Colab Notebooks/ML/семестр1/Семинар 10 - RecSys/data/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t', header=None, error_bad_lines=False, nrows=500000)

In [ ]:
all_data.shape

(500000, 6)

In [ ]:
all_data.head()

,0,1,2,3,4,5
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [ ]:
user_data = pd.read_csv('./gdrive/My Drive/Colab Notebooks/ML/семестр1/Семинар 10 - RecSys/data/lastfm-dataset-1K/userid-profile.tsv', sep = '\t', error_bad_lines=False, nrows=100000)

In [ ]:
user_data.head(100)

,#id,gender,age,country,registered
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"
...,...,...,...,...,...
95,user_000096,f,25.0,Finland,"Feb 20, 2006"
96,user_000097,m,NaN,United Kingdom,"Apr 6, 2005"
97,user_000098,f,NaN,New Zealand,"Feb 3, 2007"
98,user_000099,f,22.0,Brazil,"Jun 17, 2006"


## Collaborative filtering
На основе ALS для рекомендаций

### Предобработка данных

In [ ]:
all_data.columns

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [ ]:
R = all_data.drop(all_data.columns[4], axis=1)
R = R.drop(R.columns[1:3], axis=1)
R.head()

,0,3,5
0,user_000001,Deep Dish,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,坂本龍一,Composition 0919 (Live_2009_4_15)
2,user_000001,坂本龍一,Mc2 (Live_2009_4_15)
3,user_000001,坂本龍一,Hibari (Live_2009_4_15)
4,user_000001,坂本龍一,Mc1 (Live_2009_4_15)


In [ ]:
# Загружаем данные
R.columns = ['user', 'artist', 'plays']
R.head()

,user,artist,plays
0,user_000001,Deep Dish,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,坂本龍一,Composition 0919 (Live_2009_4_15)
2,user_000001,坂本龍一,Mc2 (Live_2009_4_15)
3,user_000001,坂本龍一,Hibari (Live_2009_4_15)
4,user_000001,坂本龍一,Mc1 (Live_2009_4_15)


In [ ]:
# Удаляем нулевые данные
R = R.dropna()
# Drop any rows that have 0 plays
R = R.loc[R.plays != 0]

In [ ]:
# Делаем названия артистов числовыми
R['user_id'] = R['user'].astype("category").cat.codes
R['artist_id'] = R['artist'].astype("category").cat.codes
R['play_id'] = R['plays'].astype("category").cat.codes

In [ ]:
# Запоминаем изначальные названия
item_lookup_artist = R[['artist_id', 'artist']].drop_duplicates()
item_lookup_artist['artist_id'] = item_lookup_artist.artist_id.astype(str)

item_lookup_plays = R[['play_id', 'plays']].drop_duplicates()
item_lookup_plays['play_id'] = item_lookup_plays.play_id.astype(str)

In [ ]:
R = R.drop(['user', 'artist', 'plays'], axis=1)

In [ ]:
# Списки уникальных номеров каждой из калонок
users = list(np.sort(R.user_id.unique()))
artists = list(np.sort(R.artist_id.unique()))
plays = list(R.play_id)

rows = R.user_id.astype(int)
cols = R.artist_id.astype(int)

In [ ]:
R

In [ ]:
# Превращаем в тип: разреженная матрица
R_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))

### Реализуем ALS
[Статья с теоретическим обоснованием метода](http://yifanhu.net/PUB/cf.pdf)

In [ ]:
def implicit_als(sparse_data, alpha_val=1, iterations=10, lambda_val=0.1, features=10):

    # alpha_val - добавляем уверенности большим данным
    confidence = sparse_data * alpha_val
    
    user_size, item_size = sparse_data.shape

    # Инициализируем случайными числами
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)

    # lambda_val - коэффициент регуляризации
    I = sparse.eye(features)
    lI = lambda_val * I
 
    # Далее ALS
    for i in range(iterations):
        print ('iteration %d of %d' % (i+1, iterations)) 
        yTy = 
        xTx = 

        for u in range(user_size):

            u_row = 
            
            # Те кто совсем не слушали группу, у тех р=0, у тех, кто слушал, распределено от 1 и выше
            p_u = u_row.copy()
            p_u

            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            # Просто так обратить не получится, используем spsolve
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)
    
        for i in range(item_size):

            i_row = confidence[:,i].T.toarray()
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y

In [ ]:
user_vecs, item_vecs = implicit_als(R_sparse, iterations=10, features=20, alpha_val=1)

iteration 1 of 10
iteration 2 of 10
iteration 3 of 10
iteration 4 of 10
iteration 5 of 10
iteration 6 of 10
iteration 7 of 10
iteration 8 of 10
iteration 9 of 10
iteration 10 of 10


###  Найдем похожих исполнителей

In [ ]:
item_lookup_artist[item_lookup_artist.artist == "Red Hot Chili Peppers"]

,artist_id,artist
13359,9811,Red Hot Chili Peppers


In [ ]:
item_lookup_artist[item_lookup_artist.artist == "Eminem"]

,artist_id,artist
90151,3980,Eminem


In [ ]:

item_id = 9811

# Get the item row
item_vec = item_vecs[item_id].T

# Calculate the similarity score
scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
top_10 = np.argsort(scores)[::-1][:10]

artists = []
artist_scores = []

# Get and print the actual artists names and scores
for idx in top_10:
    artists.append(item_lookup_artist.artist.loc[item_lookup_artist.artist_id == str(idx)].iloc[0])
    artist_scores.append(scores[idx])

similar = pd.DataFrame({'artist': artists, 'score': artist_scores})

print(similar)

                  artist       score
0               Coldplay  292.671047
1              Radiohead  263.164332
2  Red Hot Chili Peppers  251.409583
3      The White Stripes  243.217045
4        Franz Ferdinand  230.262609
5               The Cure  226.201642
6                    Air  223.886146
7             The Smiths  222.396857
8            David Bowie  218.252076
9               Gorillaz  216.867156


### Дадим рекомендации пользователю

In [ ]:
user_id = 2

# Let's print out what the user has listened to
consumed_idx = R_sparse[user_id,:].nonzero()[1].astype(str)
consumed_items = item_lookup_artist.loc[item_lookup_artist.artist_id.isin(consumed_idx)]
print(consumed_items)

      artist_id                  artist
0          3107               Deep Dish
14        13517              Underworld
16         4012         Ennio Morricone
255       10206                Röyksopp
1425       9705               Radiohead
...         ...                     ...
93441     13779                 Wedding
93474      8834             Odd Project
93553     13402            Tsunami Bomb
93557      5257            Hellogoodbye
93571      4706  Gatsbys American Dream

[935 rows x 2 columns]


In [ ]:
def recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup, num_items=10):
  
    # Get all interactions by the user
    user_interactions = 

    # We don't want to recommend items the user has consumed. So let's
    # set them all to 0 and the unknowns to 1.
    user_interactions = 
    user_interactions

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = 

    # Let's scale our scores between 0 and 1 to make it all easier to interpret.
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions*rec_vector_scaled
   
    # Get all the artist indices in order of recommendations (descending) and
    # select only the top "num_items" items. 
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    artists = []
    scores = []

    # Loop through our recommended artist indicies and look up the actial artist name
    for idx in item_idx:
        artists.append(item_lookup.artist.loc[item_lookup.artist_id == str(idx)].iloc[0])
        scores.append(recommend_vector[idx])

    # Create a new dataframe with recommended artist names and scores
    recommendations = pd.DataFrame({'artist': artists, 'score': scores})
    
    return recommendations

# Let's generate and print our recommendations
recommendations = recommend(user_id, R_sparse, user_vecs, item_vecs, item_lookup_artist)
print(recommendations)

      artist_id                  artist
0          3107               Deep Dish
14        13517              Underworld
16         4012         Ennio Morricone
255       10206                Röyksopp
1425       9705               Radiohead
...         ...                     ...
93441     13779                 Wedding
93474      8834             Odd Project
93553     13402            Tsunami Bomb
93557      5257            Hellogoodbye
93571      4706  Gatsbys American Dream

[935 rows x 2 columns]
            artist     score
0        [Unknown]  1.000000
1         The Cure  0.879532
2       Jamiroquai  0.792819
3  Michael Jackson  0.770737
4             Muse  0.730602
5    Nancy Sinatra  0.718788
6            Björk  0.701257
7      Marvin Gaye  0.661910
8      Angie Stone  0.653296
9              Rza  0.642275


## Источники и полезные ссылки
* [ALS Implicit Collaborative Filtering](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)
* [Datacamp RecSys](https://www.datacamp.com/community/tutorials/recommender-systems-python)
